# Import des Data

In [3]:
import pandas as pd

In [1]:
chanel_5k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel_5k.csv",sep=";", error_bad_lines=False)
chanel = chanel_5k[chanel_5k['language'] == 'en']['text'].tolist()
publication_time_chanel = chanel_5k[chanel_5k['language'] == 'en']['publication_time'].tolist()
len(chanel)

C:\Users\a.tekiouk\AppData\Local\Temp\ipykernel_4488\3904026442.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chanel_5k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel_5k.csv",sep=";", error_bad_lines=False)


2285

# Modification de la pipe

In [4]:
import numpy as np
import spacy

nlp = spacy.load("en_core_web_sm")

from spacymoji import Emoji


nlp.add_pipe("emoji", first=True)

In [5]:
import spacy 
from spacy.vocab import Vocab
from spacy.language import Language
from spacy.tokens import Token


@Language.component("hashtag")
def hashtag_pipe(doc):
    merged_hashtag = False
    while True:
        for token in doc:
            if token.text == '#':
                if token.head is not None:
                    start_index = token.i
                    end_index = start_index + 1
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[start_index:end_index+1])
                        merged_hashtag = True
                        break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc


In [6]:
nlp.add_pipe("hashtag", first=True)
hashtag_getter = lambda token: token.text[0] in ("#")
Token.set_extension("is_hashtag", getter=hashtag_getter, force=True)

# Métohdes

In [7]:
def top_hashtags(text, N, top):
    hashtag = []
    index_vector = np.random.randint(len(text), size=N)
    for i in index_vector:
        doc = nlp(text[i])
        for token in doc:
            if token._.is_hashtag:
                hashtag.append(token.text)
    Hashtag_corpus = pd.DataFrame()
    Hashtag_corpus['hashtag'] = hashtag
    Hashtag_corpus2 = Hashtag_corpus.groupby("hashtag").size()
    print(Hashtag_corpus2.nlargest(top).tail(top))

In [8]:
def top_emojis(text, N, top):
    emoji = []
    index_vector = np.random.randint(len(text), size=N)
    for i in index_vector:
        doc = nlp(text[i])
        for token in doc:
            if token._.is_emoji:
                emoji.append(token.text)
    Emoji_corpus = pd.DataFrame()
    Emoji_corpus['emoji'] = emoji
    Emoji_corpus2 = Emoji_corpus.groupby("emoji").size()
    print(Emoji_corpus2.nlargest(top).tail(top))

In [10]:
def word_ratio(df):
    ratio = []
    p=0
    for i in df['text']:
        doc = nlp(i)
        nb_word = 0
        nb_hash = 0
        for token in doc:
            if(token._.is_hashtag):
                nb_hash+=1
            else:
                nb_word+=1
        ratio.append(nb_word/(nb_hash+nb_word))
        print(p)
        p=p+1
    df['ratio_word'] = ratio
    return df

In [11]:
def caps_ratio(df):
    ratio = []
    for i in df['text']:
        doc = nlp(i)
        nb_lower = 0
        nb_caps = 0
        for token in doc:
            if(token.text.isupper()):
                nb_caps+=1
            else:
                nb_lower+=1
        ratio.append(nb_caps/(nb_caps+nb_lower))
    df['ratio_caps'] = ratio
    return df

In [12]:
import re

def clean_hashtag(t): #--text
    hashtag_pattern= re.compile("#[A-Za-z0-9_]+")
    return re.sub(hashtag_pattern,"", t) #On supprime tout les types de #


def del_double(txt,publication_time,s,method): # --text --liste des date et heure de publication pour chaque élément de la liste --seuil (s dans [0;1] ou [0;100] selon la méthode) --méthode
    t = txt.copy()
    distance = method #initialisiation de levenshtein avec la distance normalisée.
    i = 0
    r = len(t)
    while(i<r):
        r = len(t)
        j=i+1
        while(j<r):
            if(distance(clean_hashtag(t[i]).strip(),clean_hashtag(t[j]).strip()) <= s ): # Si la distance entre les deux élemens de la liste inf à seuil
                if(publication_time[i]<publication_time[j]):
                    del t[j] #delete
                    r = len(t) #on actualise la taille de la listes
                else:
                    del t[i]
                    r = len(t) #on actualise la taille de la listes
            else:
                j+=1
        i+=1
    return t

# Creation d'un sample de validation

In [11]:
import random
chanel_junk_valid = random.choices(chanel, k=700)

NameError: name 'chanel' is not defined

In [17]:
import textdistance
dist = textdistance.levenshtein.normalized_distance
chanel_junk_valid_dd = del_double(chanel_junk_valid,publication_time_chanel,0.5,dist)

In [18]:
len(chanel_junk_valid_dd)

207

In [19]:
chanel_junk_valid_df= pd.DataFrame()
chanel_junk_valid_df['text'] = chanel_junk_valid_dd

In [20]:
chanel_junk_valid_df.to_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel_junk_valid.csv")
# Ajout de la variable is_junk sur excel

In [13]:
chanel_junk_valid_new = pd.read_excel('C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel_junk_valid_new.xlsx')
chanel_junk_valid_new = chanel_junk_valid_new[['text','is_junk']]

In [14]:
chanel_junk_valid_new.head()

,text,is_junk
0,The camelia - a long time symbol of Chanel and...,0
1,Wknds are my happy place 🤍 even if they do inc...,1
2,Used Card holder XL 24999.-\n\nLine: 244xfzmn,1
3,My love for a chanelofficial classic flap 😌| #...,0
4,HAPPY BIRTHDAY KING KUMA\n\nJENNIE #SOLO STAGE...,1


# Ajout de features

In [65]:
chanel_junk_valid_new_temp = chanel_junk_valid_new.drop(chanel_junk_valid_new.index[102])

### Ratio de mot

In [66]:
word_ratio(chanel_junk_valid_new_temp)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


KeyboardInterrupt: 

In [ ]:
chanel_junk_valid_new.head()

### Ratio de caps

In [ ]:
caps_ratio(chanel_junk_valid_new)

In [20]:
chanel_junk_valid_new.head()

,text,is_junk
0,The camelia - a long time symbol of Chanel and...,0
1,Wknds are my happy place 🤍 even if they do inc...,1
2,Used Card holder XL 24999.-\n\nLine: 244xfzmn,1
3,My love for a chanelofficial classic flap 😌| #...,0
4,HAPPY BIRTHDAY KING KUMA\n\nJENNIE #SOLO STAGE...,1


### Top hashtags

In [ ]:
top_hashtags(chanel_junk_valid_new)

### Top emojis

In [ ]:
top_emojis(chanel_junk_valid_new)